In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import sqlite3

%matplotlib inline

In [ ]:
# Create a connection to the SQLite database
conn = sqlite3.connect('../Data/db/fraude_detection_warehouse.db')

In [ ]:
transactions = pd.read_sql('SELECT * FROM transactions', conn)
transactions.head()

In [ ]:
transactions.info()

In [ ]:
transactions.describe(include="all")

changing date type

In [ ]:
transactions.transaction_date = pd.to_datetime(transactions.transaction_date)
transactions.info()

### transaction_type

In [ ]:
sns.countplot(transactions, x="transaction_type", hue='transaction_type')

In [ ]:
transactions.transaction_type.value_counts().sort_values()

In [ ]:
length = transactions.shape[0]
# create dataframe percentage of transaction type
proportion = pd.DataFrame(transactions['transaction_type'].value_counts(normalize=True)*100)
proportion

### transaction amount

In [ ]:
# transaction amount by transaction type
pd.DataFrame(transactions.groupby('transaction_type').amount.aggregate(['mean', 'median']))

In [ ]:
transfert = transactions[transactions.transaction_type == 'transfert']
depot = transactions[transactions.transaction_type == 'depôt']
retrait = transactions[transactions.transaction_type == 'retrait']
type_list = [transfert, depot, retrait]


In [ ]:
sns.histplot(transactions, x="amount", kde=True)

In [ ]:
plt.figure(figsize=(15,7))
for i, value in enumerate(type_list, 1):
    plt.subplot(2,2,i)
    sns.histplot(value, x='amount', kde=True)
    plt.title(value.transaction_type.iloc[0])
    plt.tight_layout()
    plt.plot()

In [ ]:
plt.figure(figsize=(15,4))
for i, value in enumerate(type_list, 1):
    plt.subplot(2,2,i)
    sns.boxplot(value, x='amount')
    plt.title(value.transaction_type.iloc[0])
    plt.tight_layout()
    plt.plot()

### Analyse temporelle

In [ ]:
transactions['month'] = pd.to_datetime(transactions['transaction_date'].dt.strftime('%Y-%m'))
transactions['week'] = transactions['transaction_date'].dt.isocalendar().week
transactions['month_number'] = transactions['transaction_date'].dt.month
transactions['year'] = transactions['transaction_date'].dt.year

In [ ]:
transactions_copy = transactions.set_index('transaction_date')
transactions_copy.head()

Montant de transaction par jour

In [ ]:
transactions_per_day = pd.DataFrame(transactions_copy.groupby('transaction_date').amount.aggregate("mean")).reset_index()
fig = px.line(transactions_per_day, x='transaction_date', y='amount', title='transactions_per_day amount')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

Montant de transaction par mois

In [ ]:
transactions_per_month = pd.DataFrame(transactions_copy.groupby("month").amount.aggregate('mean')).reset_index()
fig = px.line(transactions_per_month, x='month', y='amount', title='transactions_per_month amount')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

Montant de transaction par an

In [ ]:
transactions_per_month = pd.DataFrame(transactions_copy.groupby("year").amount.aggregate('mean')).reset_index()
fig = px.line(transactions_per_month, x='year', y='amount', title='transactions_per_month amount')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

Nombre de transaction par jour

In [ ]:
transactions_per_day_nbr = pd.DataFrame(transactions_copy.groupby("transaction_date").transaction_id.count()).reset_index()
fig = px.line(transactions_per_day_nbr, x='transaction_date', y='transaction_id', title='transactions_per_day_nbr')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

Nombre de transaction par mois

In [ ]:
transactions_per_month_nbr = pd.DataFrame(transactions_copy.groupby("month").transaction_id.count()).reset_index()
fig = px.line(transactions_per_month_nbr, x='month', y='transaction_id', title='transactions_per_month_nbr')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

Nombre de transaction par an

In [ ]:
transactions_per_month_nbr = pd.DataFrame(transactions_copy.groupby("year").transaction_id.count()).reset_index()
fig = px.line(transactions_per_month_nbr, x='year', y='transaction_id', title='transactions_per_month_nbr')

fig.update_xaxes(rangeslider_visible=True)

## DISTRIBUTION DES TYPES D'APPAREILS UTILISES ##

In [ ]:
devices = pd.read_sql(" SELECT * FROM DEVICES ", conn)
devices.info()

In [ ]:
plt.figure(figsize=(6,4))
devices['device_type'].value_counts().plot(kind='bar',color= 'green')

plt.title('DISTRIBUTION DES TYPES D\'APPAREILS UTILISES')
plt.xlabel('type d\'appareils')
plt.ylabel('nombre d\'appareils')
plt.show()

## Distribution des systèmes d'exploitation ##

In [ ]:
df=devices['os']
df

In [ ]:
plt.figure(figsize=(5,4))
devices['os'].value_counts().plot(kind='pie', autopct='%1.1f%%')

plt.title('Distribution des systèmes d\'exploitation')
#plt.xlabel('type d\'os')
#plt.ylabel('nombre d\'os')
plt.show()

## Nombre de transactions par appareil ##

In [ ]:
data= pd.read_sql("SELECT device_id, COUNT(transaction_id) as total_transactions FROM transactions GROUP BY device_id", conn)
data.info()
data.head()

In [ ]:
data_limited = data.head(10)  #limiter les donnees aux 20 premieres lignes de la table
plt.figure(figsize=(10,6))

plt.bar(data_limited['device_id'], data_limited['total_transactions'], color='orange')

plt.title('Nombre de transactions par appareil')
plt.xlabel(' ID appareils')
plt.ylabel('nombre de  transactions')
plt.xticks(rotation=90)
plt.show()

## Montant total des transactions par appareil ##

In [ ]:
Total_transaction= pd.read_sql("SELECT device_id, SUM(amount) as total_amount FROM transactions GROUP BY device_id", conn)

Total_transaction.head()

In [ ]:
data_limited = Total_transaction.head(10)  #limiter les donnees aux 20 premieres lignes de la table
plt.figure(figsize=(10,6))

plt.bar(data_limited['device_id'], data_limited['total_amount'], color='orange')

plt.title('Nombre de transactions par appareil')
plt.xlabel(' ID appareils')
plt.ylabel('nombre de  transactions')
plt.xticks(rotation=90)
plt.show()

# ANALYSE DES CLIENTS #

## Nombre total de clients actifs (client actif= plus de 3 transactions) ##

In [ ]:
Total_clients= pd.read_sql("SELECT  COUNT (*) as active_clients_count FROM (SELECT customer_id FROM transactions GROUP BY customer_id HAVING COUNT(transaction_id) >= 3) as total_customers ", conn)
Total_clients.head()

In [ ]:
clients= pd.read_sql("SELECT * from customers", conn)

clients.head()

## Distribution des âges des clients ##

In [ ]:
# ajout de la colone age
cursor=conn.cursor()
cursor.execute("ALTER TABLE customers ADD COLUMN age ")

#calcul des ages
cursor.execute("""
               UPDATE customers  SET age= (strftime('%Y' , 'now')- strftime('%Y' , date_of_birth ))- 
                                        (strftime('%m-%d', 'now')< strftime('%m-%d', date_of_birth));
               """)
conn.commit()

In [ ]:
age= pd.read_sql("SELECT * FROM customers", conn)
age.head()

In [ ]:
age= pd.read_sql("SELECT age, COUNT(*) as age_count FROM customers GROUP BY age ", conn)
age.head(10)

In [ ]:
age_limited= age.head(20)
plt.figure(figsize=(10,6))

plt.bar(age_limited['age'], age_limited['age_count'], color='orange')

plt.title('Nombre de transactions par appareil')
plt.xlabel(' age')
plt.ylabel('nombre ')
plt.xticks(rotation=90)
plt.show()

## Nombre moyen de transactions par client ##

In [ ]:
moyenne_transaction= pd.read_sql("SELECT customer_id, COUNT(transaction_id) AS num_transaction FROM transactions GROUP BY customer_id", conn)
moyenne_transaction.head()

In [ ]:
moyenne_transaction_limited= moyenne_transaction.head(20)
plt.figure(figsize=(10,6))

plt.bar(moyenne_transaction_limited['customer_id'], moyenne_transaction_limited['num_transaction'], color='orange')

plt.title('Nombre moyen de transactions par client')
plt.xlabel(' ID client')
plt.ylabel('nombre moyen  de  transactions')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Étape 2: Trouver le nombre maximum de transactions
max_transactions = moyenne_transaction['num_transaction'].max()

In [ ]:
# Étape 3: Trouver les clients ayant le maximum de transactions
clients_max_transactions = pd.read_sql(f"""
    SELECT c.customer_id, c.age
    FROM customers c
    JOIN (
        SELECT customer_id 
        FROM transactions 
        GROUP BY customer_id  
        HAVING COUNT(transaction_id) = {max_transactions}
    ) t ON c.customer_id = t.customer_id
""", conn)

clients_max_transactions

## Montant moyen des transactions par client. ##

In [ ]:
montant_moyen= pd.read_sql("SELECT customer_id, AVG(amount) AS avg_amount FROM transactions GROUP BY customer_id", conn)
montant_moyen.head()

In [ ]:
montant_moyen_limited=montant_moyen.head(20)
plt.figure(figsize=(10,6))

plt.bar(montant_moyen_limited['customer_id'], montant_moyen_limited['avg_amount'], color=plt.cm.Paired(range(len(montant_moyen_limited))))

plt.title('montant moyen des transactions par client')
plt.xlabel('id clients')
plt.ylabel('avg_amount')
plt.xticks(rotation=90)
plt.show()

# Alertes de Fraude #

## Nombre total d'alertes générées ##

In [ ]:
alertes= pd.read_sql("SELECT COUNT(alert_id) as alert FROM alerts", conn)
alertes

## Distribution des types d'alertes ##

In [ ]:
distribution_alert= pd.read_sql("SELECT alert_type, COUNT(*) as nbre_alerte FROM alerts GROUP BY alert_type", conn )
distribution_alert.head()

In [ ]:
plt.figure(figsize=(10,6))

plt.bar(distribution_alert['alert_type'], distribution_alert['nbre_alerte'], color=plt.cm.Paired(range(len(distribution_alert))))

plt.title('Distribution des types d\'alertes ')
plt.xlabel('type alerte')
plt.ylabel('nombre')

## Taux de résolution des alertes(nombre d'alertes fermées divisé par le nombre total d'alertes). ##

In [ ]:
alerte_fermee=   pd.read_sql("SELECT COUNT(alert_status)  FROM alerts WHERE alert_status='closed' ", conn )
alerte_fermee

In [ ]:
total_alerte= pd.read_sql("SELECT COUNT(alert_id)  FROM alerts ", conn)
total_alerte

In [ ]:
taux_resolution= (alerte_fermee['COUNT(alert_status)'][0] / total_alerte['COUNT(alert_id)'][0])*100
print(taux_resolution)

## Temps moyen de résolution des alertes ##